In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
from concurrent.futures import ThreadPoolExecutor
import time
import requests
import numpy as np

In [2]:
# load json data
df = json_normalize(pd.Series(open('enriched_articles.json').readlines()).apply(json.loads))

<ipython-input-2-8e6e011e5c4e>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(pd.Series(open('enriched_articles.json').readlines()).apply(json.loads))


In [3]:
# unpack a column of lists into rows, return new dataframe
def json_unpack(dataframe,column,name):
    df_single = dataframe[column].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame(name).join(dataframe[['url']], how='left')
    df_single.reset_index(inplace=True,drop=True)
    df_new=json_normalize(df_single[name].tolist())
    df_new.reset_index(drop=True,inplace=True)
    result_df=pd.concat([df_new,df_single],axis=1)
    return  result_df

In [4]:
# unpack entity column, get df_entity
df_entity=json_unpack(df,'entities','entity')

<ipython-input-3-cf2b8c0b0ffd>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_single = dataframe[column].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame(name).join(dataframe[['url']], how='left')
<ipython-input-3-cf2b8c0b0ffd>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_new=json_normalize(df_single[name].tolist())


In [5]:
df_entity

,id,type,matchingTokens,entityId,freebaseTypes,confidenceScore,wikiLink,matchedText,freebaseId,relevanceScore,...,startingPos,endingPos,wikidataId,unit,crunchbaseId,lei,permid,figi,entity,url
0,0,"[ChemicalSubstance, ChemicalCompound]",[563],Water,"[/exhibitions/exhibition_subject, /chemistry/m...",2.9090,http://en.wikipedia.org/wiki/Water,water,/m/0838f,0.3927,...,2809,2814,Q283,NaN,NaN,NaN,NaN,NaN,"{'id': 0, 'type': ['ChemicalSubstance', 'Chemi...",https://thewaterproject.org/water-crisis/water...
1,1,NaN,[576],Floor,[/organization/organization_sector],0.9774,http://en.wikipedia.org/wiki/Floor,Floor,/m/0l7_8,0.1684,...,2865,2870,Q217164,NaN,NaN,NaN,NaN,NaN,"{'id': 1, 'matchingTokens': [576], 'entityId':...",https://thewaterproject.org/water-crisis/water...
2,2,"[Place, PopulatedPlace, Settlement, City]","[578, 579, 580]","Concord, New Hampshire","[/location/statistical_region, /location/cityt...",1.7690,"http://en.wikipedia.org/wiki/Concord,_New_Hamp...","Concord, NH",/m/01x96,0.1524,...,2873,2884,Q28249,NaN,NaN,NaN,NaN,NaN,"{'id': 2, 'type': ['Place', 'PopulatedPlace', ...",https://thewaterproject.org/water-crisis/water...
3,3,"[Place, PopulatedPlace, Continent]",[91],Africa,"[/tv/tv_location, /location/dated_location, /m...",11.6100,http://en.wikipedia.org/wiki/Africa,Africa,/m/0dg3n1,0.3734,...,465,471,Q15,NaN,NaN,NaN,NaN,NaN,"{'id': 3, 'type': ['Place', 'PopulatedPlace', ...",https://thewaterproject.org/water-crisis/water...
4,4,"[Place, PopulatedPlace, Country]",[584],United States,"[/film/film, /media_common/adaptation, /tv/tv_...",39.9200,http://en.wikipedia.org/wiki/United_States,USA,/m/09c7w0,0.2136,...,2895,2898,Q30,NaN,NaN,NaN,NaN,NaN,"{'id': 4, 'type': ['Place', 'PopulatedPlace', ...",https://thewaterproject.org/water-crisis/water...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72838,50,[Company],[229],Sanyati,[/organization/organization],0.5000,,Sanyati,NaN,0.0000,...,1196,1203,NaN,NaN,NaN,NaN,NaN,NaN,"{'id': 50, 'type': ['Company'], 'matchingToken...",https://www.herald.co.zw/council-steps-up-effo...
72839,51,[Number],[123],6,NaN,0.5000,,6,NaN,0.0000,...,657,658,NaN,NaN,NaN,NaN,NaN,NaN,"{'id': 51, 'type': ['Number'], 'matchingTokens...",https://www.herald.co.zw/council-steps-up-effo...
72840,52,[Number],[127],7,NaN,0.5000,,7,NaN,0.0000,...,672,673,NaN,NaN,NaN,NaN,NaN,NaN,"{'id': 52, 'type': ['Number'], 'matchingTokens...",https://www.herald.co.zw/council-steps-up-effo...
72841,53,[Number],[134],4,NaN,0.5000,,4,NaN,0.0000,...,699,700,NaN,NaN,NaN,NaN,NaN,NaN,"{'id': 53, 'type': ['Number'], 'matchingTokens...",https://www.herald.co.zw/council-steps-up-effo...


In [6]:
# write entity column json data into different columns
entity=json_normalize(df_entity['entity'].tolist())
entity.reset_index(drop=True,inplace=True)
entity

<ipython-input-6-0bce73ce25fe>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  entity=json_normalize(df_entity['entity'].tolist())


,id,type,matchingTokens,entityId,freebaseTypes,confidenceScore,wikiLink,matchedText,freebaseId,relevanceScore,entityEnglishId,startingPos,endingPos,wikidataId,unit,crunchbaseId,lei,permid,figi
0,0,"[ChemicalSubstance, ChemicalCompound]",[563],Water,"[/exhibitions/exhibition_subject, /chemistry/m...",2.9090,http://en.wikipedia.org/wiki/Water,water,/m/0838f,0.3927,Water,2809,2814,Q283,NaN,NaN,NaN,NaN,NaN
1,1,NaN,[576],Floor,[/organization/organization_sector],0.9774,http://en.wikipedia.org/wiki/Floor,Floor,/m/0l7_8,0.1684,Floor,2865,2870,Q217164,NaN,NaN,NaN,NaN,NaN
2,2,"[Place, PopulatedPlace, Settlement, City]","[578, 579, 580]","Concord, New Hampshire","[/location/statistical_region, /location/cityt...",1.7690,"http://en.wikipedia.org/wiki/Concord,_New_Hamp...","Concord, NH",/m/01x96,0.1524,"Concord, New Hampshire",2873,2884,Q28249,NaN,NaN,NaN,NaN,NaN
3,3,"[Place, PopulatedPlace, Continent]",[91],Africa,"[/tv/tv_location, /location/dated_location, /m...",11.6100,http://en.wikipedia.org/wiki/Africa,Africa,/m/0dg3n1,0.3734,Africa,465,471,Q15,NaN,NaN,NaN,NaN,NaN
4,4,"[Place, PopulatedPlace, Country]",[584],United States,"[/film/film, /media_common/adaptation, /tv/tv_...",39.9200,http://en.wikipedia.org/wiki/United_States,USA,/m/09c7w0,0.2136,United States,2895,2898,Q30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72838,50,[Company],[229],Sanyati,[/organization/organization],0.5000,,Sanyati,NaN,0.0000,,1196,1203,NaN,NaN,NaN,NaN,NaN,NaN
72839,51,[Number],[123],6,NaN,0.5000,,6,NaN,0.0000,,657,658,NaN,NaN,NaN,NaN,NaN,NaN
72840,52,[Number],[127],7,NaN,0.5000,,7,NaN,0.0000,,672,673,NaN,NaN,NaN,NaN,NaN,NaN
72841,53,[Number],[134],4,NaN,0.5000,,4,NaN,0.0000,,699,700,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Use wikidata api to get the country from a populated_place entity
failed=[]
def get_country(ent_wiki_id):
    url='https://www.wikidata.org/w/api.php?action=wbgetentities&ids='+ent_wiki_id+'&props=claims&languages=en&format=json'
    try:
        response = requests.get(url,timeout=(10,20))
        ent_data = response.json()
#       get wikidata id for the country where the entity locates, P17 is country property in wikidata
        country_id = ent_data['entities'][ent_wiki_id]['claims']['P17'][0]['mainsnak']['datavalue']['value']['id']
    except:
        failed.append(ent_wiki_id)
    else:
        url2='https://www.wikidata.org/w/api.php?action=wbgetentities&ids='+country_id+'&props=labels&languages=en&format=json'
        try:
#           label of wikidata of the country page is the country name
            country_response = requests.get(url2,timeout=(10,20))
            country_data = country_response.json()
            country = country_data['entities'][country_id]['labels']['en']['value']
        except:
            failed.append(ent_wiki_id)
        else:
            return country

In [15]:
entity

,id,type,matchingTokens,entityId,freebaseTypes,confidenceScore,wikiLink,matchedText,freebaseId,relevanceScore,entityEnglishId,startingPos,endingPos,wikidataId,unit,crunchbaseId,lei,permid,figi
0,0,"[ChemicalSubstance, ChemicalCompound]",[563],Water,"[/exhibitions/exhibition_subject, /chemistry/m...",2.9090,http://en.wikipedia.org/wiki/Water,water,/m/0838f,0.3927,Water,2809,2814,Q283,NaN,NaN,NaN,NaN,NaN
1,1,NaN,[576],Floor,[/organization/organization_sector],0.9774,http://en.wikipedia.org/wiki/Floor,Floor,/m/0l7_8,0.1684,Floor,2865,2870,Q217164,NaN,NaN,NaN,NaN,NaN
2,2,"[Place, PopulatedPlace, Settlement, City]","[578, 579, 580]","Concord, New Hampshire","[/location/statistical_region, /location/cityt...",1.7690,"http://en.wikipedia.org/wiki/Concord,_New_Hamp...","Concord, NH",/m/01x96,0.1524,"Concord, New Hampshire",2873,2884,Q28249,NaN,NaN,NaN,NaN,NaN
3,3,"[Place, PopulatedPlace, Continent]",[91],Africa,"[/tv/tv_location, /location/dated_location, /m...",11.6100,http://en.wikipedia.org/wiki/Africa,Africa,/m/0dg3n1,0.3734,Africa,465,471,Q15,NaN,NaN,NaN,NaN,NaN
4,4,"[Place, PopulatedPlace, Country]",[584],United States,"[/film/film, /media_common/adaptation, /tv/tv_...",39.9200,http://en.wikipedia.org/wiki/United_States,USA,/m/09c7w0,0.2136,United States,2895,2898,Q30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72838,50,[Company],[229],Sanyati,[/organization/organization],0.5000,,Sanyati,NaN,0.0000,,1196,1203,NaN,NaN,NaN,NaN,NaN,NaN
72839,51,[Number],[123],6,NaN,0.5000,,6,NaN,0.0000,,657,658,NaN,NaN,NaN,NaN,NaN,NaN
72840,52,[Number],[127],7,NaN,0.5000,,7,NaN,0.0000,,672,673,NaN,NaN,NaN,NaN,NaN,NaN
72841,53,[Number],[134],4,NaN,0.5000,,4,NaN,0.0000,,699,700,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
entity_place=(pd.concat([df_entity.loc[:,'url'],entity],axis=1)
#                filter out none values
                 .query("type==type")
                 .assign(place_filter = lambda x: x['type'].astype('str'))
# #                only use the PopulatedPlace entity type
                 .query("place_filter.str.contains('PopulatedPlace',case=False)",engine='python')
# #                get country from the entity wikidataid
                 .assign(wiki_country = lambda x : x['wikidataId'].apply(lambda i:get_country(i)))
                )
           
entity_place.reset_index(inplace=True,drop=True)


In [24]:
# retry the failed requests
entity_place.loc[entity_place['wikidataId'].isin(failed),'wiki_country']=entity_place.loc[entity_place['wikidataId'].isin(failed),'wikidataId'].apply(lambda x: get_country(x))

In [25]:
# check for where wikidata couldn't find its country
entity_place[entity_place['wiki_country'].isna()]['entityId'].unique()

array(['Africa', 'Urban area', 'Village', 'Europe', 'African Union',
       'European Union', 'United Nations', 'Administrative centre',
       'Well', 'Asia', 'Manyoni', 'Allies of World War II',
       'Municipal council', 'Human settlement', 'Somaliland',
       'Allies of World War I', 'Italian Somaliland', 'Tunnel', 'Kiosk',
       'Lumber', 'Suburb', 'Hadiya Sultanate', 'Aurora',
       'Islamic State of Iraq and the Levant', 'Middle East',
       'Landlocked country', 'Arab League', 'Arab world', 'Ancient Egypt',
       'North Africa', 'Central business district', 'Hanseatic League',
       'Municipality', 'Province', 'Commuter town', 'Glen View, Harare',
       'Central Asia', 'Zimbabwe', 'Caribbean', 'Commonwealth of Nations',
       'South America', 'Community of Portuguese Language Countries',
       'Latin America', 'Common Market for Eastern and Southern Africa'],
      dtype=object)

In [26]:
# Manually fix Somaliland
entity_place.loc[(entity_place['entityId']=='Somaliland')|(entity_place['entityId']=='somaliland'),'wiki_country'] = 'Somaliland'

In [27]:
# aggregate the same country together, get count
def group_by_country (df):
    country_group = df.groupby(['url','wiki_country']).id.count().to_frame()
    country_group.reset_index(inplace=True)       
    return country_group

In [28]:
# create countries list by article
def group_by_article (df):
    df2 = df.groupby('url').country_weight.apply(list).to_frame()
    df2.reset_index(inplace=True)
    return df2

In [29]:
result_country=(entity_place .pipe(group_by_country)
# country_weight is a tuple of the country and the times the country is mentioned using the results from wikidata
                             .assign(country_weight = lambda x : list(zip(x.wiki_country, x.id)))
                             .pipe(group_by_article))

In [30]:
# write all country query results to countries.csv
result_country.to_csv('countries.csv')

In [31]:
# unpack topic column, get df_topic
df_topic=json_unpack(df,'topics','topic_new')

<ipython-input-3-cf2b8c0b0ffd>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_single = dataframe[column].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame(name).join(dataframe[['url']], how='left')
<ipython-input-3-cf2b8c0b0ffd>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_new=json_normalize(df_single[name].tolist())


In [53]:
# all articles with climate change as topic
climate_urls=df_topic[df_topic['label'].str.contains('climate',case=False)&df_topic['label'].str.contains('change',case=False)]['url'].unique()
climate_articles=df_topic[df_topic['url'].isin(climate_urls)]
climate_articles.reset_index(inplace=True,drop=True)

# all topics include str 'climate' and 'change'
climate_exact_topics=df_topic[df_topic['label'].str.contains('climate',case=False)&df_topic['label'].str.contains('change',case=False)]['label'].unique()

In [61]:
# manually filter out topics relevant to climate change in climate_articles, who doesn't have the word 'climate change' in it
# get list climate_general_topics
relevant_words="disaster|global warming|mudflow|rain|landslide|cyclone|vortices|flood|hurricane|storm|disease|drought|dry season|food security|water scarcity|forced migration|weather hazards|storm|locust|exreme weather|conflicts|famine|erosion|arid|desertification|land degradation|deforestation|pest|severe weather|Cyanotoxin|algal|Cyanobacteria|Eutrophication|Greenhouse"
climate_general_topics=climate_articles[climate_articles['label'].str.contains(relevant_words,case=False)].label.unique()

In [62]:
# select only articles with or relevant to climate change as topics_exact and topics_general
# zip climate_relevant topics and climate_exact topics into two columns
df_climate=(climate_articles.groupby('url').label.apply(list).to_frame()
                            .assign(topics_exact = lambda x: x['label'].apply(lambda m: [i for i in m if i in climate_exact_topics]))
                            .assign(topics_general = lambda x: x['label'].apply(lambda m: [i for i in m if i in climate_general_topics]))
           )
df_climate.reset_index(inplace=True)
df_climate

,url,label,topics_exact,topics_general
0,http://english.ahram.org.eg/NewsContent/4/0/36...,"[Public sphere, Politics, Policy, Human activi...",[Climate change],[Global warming]
1,http://kubatana.net/2020/07/16/byo-water-crisi...,"[Bulawayo, Public sphere, Human activities, Po...",[Climate change],"[Water scarcity, Disasters, Global warming]"
2,http://new.observer.org.sz/details.php?id=10893,"[Sanitation, Drinking water, Water, WASH, Wate...","[Climate change, Climate change (general conce...","[Water scarcity, Effects of global warming, Gl..."
3,http://new.observer.org.sz/details.php?id=8333,"[Water supply, Water, Eswatini, Oxidizing agen...","[Climate change mitigation, Climate change pol...",[Global warming]
4,http://opiniojuris.org/2020/05/04/so-close-yet...,"[Ethiopia, Nile, Climate change mitigation, Tr...","[Climate change mitigation, Climate change (ge...","[Drought, Flooding of the Nile, Flood, Rain, N..."
...,...,...,...,...
131,https://www.worldvision.com.au/global-issues/w...,"[Drinking water, Water, World Vision Internati...","[Climate change (general concept), Climate cha...","[Food security, Drought, Water scarcity, Disas..."
132,https://www.wvi.org/article/relief-work-covers...,"[Drought, Office of Foreign Disaster Assistanc...","[Climate change adaptation, Climate change (ge...","[Drought, Office of Foreign Disaster Assistanc..."
133,https://www.wvi.org/stories/coronavirus-health...,"[Water, Drinking water, Natural environment, L...","[Climate change, Climate change (general conce...","[Rain, Flood, Coronavirus disease 2019, Landsl..."
134,https://www.wvi.org/stories/eswatini/minister-...,"[Water, Transparent materials, Hydrology, Natu...","[Climate change policy, Climate change, Politi...",[Global warming]


In [69]:
# use the original df to get the other columns such as title and id
dfs=[df_climate,result_country,
     (df.assign(id= lambda x: x.index).loc[:,['url','country','id','title']])]

In [71]:
# join the country df ,the topics df and the original df with url
# get final dataframe result
result= (dfs[0].set_index('url')
              .join(dfs[1].set_index('url'))
              .join(dfs[2].set_index('url'))
              .sort_values('id')
        )
result.rename(columns={'url':'article_url','label':'all_topics','id':'article_id','country_weight':'countries','country':'original_country'},inplace=True)
result.reset_index(inplace=True)

In [72]:
result

,url,all_topics,topics_exact,topics_general,countries,original_country,article_id,title
0,https://postconflict.unep.ch/publications/UNEP...,"[United Nations Environment Programme, Natural...","[Climate change, Climate change (general conce...","[Disasters, Global warming, Environmental disa...","[(South Sudan, 1), (Sudan, 1)]",DRC,6,Water Issues in the Democratic Republic of the...
1,https://www.afrik21.africa/en/drc-government-u...,"[Inga dams, Hydroelectricity, Democratic Repub...","[Climate change policy, Climate change, Politi...","[Global warming, Coronavirus disease 2019]","[(Democratic Republic of the Congo, 8), (Franc...",DRC,10,Government unites Africa around the Grand Inga...
2,https://www.afrik21.africa/en/drc-heavy-floodi...,"[Democratic Republic of the Congo, Global warm...","[Intergovernmental Panel on Climate Change, Cl...","[Global warming, Flood, Natural disasters, Dis...","[(Burundi, 1), (Central African Republic, 1), ...",DRC,11,Heavy flooding ravage the east
3,https://www.afrik21.africa/en/drc-afdb-provide...,"[Inga dams, Human activities, Economy, Democra...",[Climate change],[Global warming],"[(Democratic Republic of the Congo, 9), (Franc...",DRC,12,AfDB provides $73 million for Grand Inga Dam f...
4,https://www.unenvironment.org/news-and-stories...,"[Democratic Republic of the Congo, Water, Unit...","[Climate change, Climate change (general conce...","[Flood, Global warming, Drainage basin]","[(Democratic Republic of the Congo, 15), (Swit...",DRC,15,UNEP launches pioneering water initiative in D...
...,...,...,...,...,...,...,...,...
144,https://www.bbc.com/news/world-africa-38103849,"[Harare, Water, Drought, Liquid dielectrics, R...","[Climate change (general concept), Climate cha...","[Drought, Rain, Water scarcity, Disasters, Nat...","[(Nigeria, 10), (Zimbabwe, 28)]",Zimbabwe,409,Zimbabwe water shortages: 'I don't remember ta...
145,https://www.weforum.org/agenda/2020/01/zimbabw...,"[Bulawayo, Water, Human activities, Liquid die...","[Climate change (general concept), Climate cha...","[Drought, Water scarcity, Global warming, Coro...","[(Nigeria, 4), (Zimbabwe, 12)]",Zimbabwe,414,Severe drought is leaving this town in Zimbabw...
146,https://www.herald.co.zw/water-shortage-and-co...,"[Zimbabwe, Water, Drinking water, Human right ...","[Climate change mitigation, Climate change, Cl...","[Global warming, Disease, Coronavirus disease ...","[(Nigeria, 12), (Zimbabwe, 16)]",Zimbabwe,427,Water shortage and COVID19
147,https://www.herald.co.zw/editorial-comment-ste...,"[Water pollution, Stormwater, Sewage treatment...",[Climate change mitigation],"[Stormwater, Storm drain, Rain garden, Flood, ...","[(United States of America, 2), (Zimbabwe, 3)]",Zimbabwe,430,Editorial Comment: Step up fight against water...


In [73]:
# write into json file
data=json.loads(result.to_json(orient="records"))
with open('data.json', 'w') as f:
    json.dump(data, f)